In [ ]:
# сменим рабочую директорию на корневую
%cd ..

In [71]:
# submodules нужно скачать вручную, иначе падает с ошибкой
!cd submodules/ && \
    git clone https://github.com/Pe4enIks/BasicSR.git && \
    mv BasicSR basicsr && \
    git clone https://github.com/Pe4enIks/Real-ESRGAN.git && \
    mv Real-ESRGAN real-esrgan && \
    git clone https://github.com/Pe4enIks/ResShift && \
    mv ResShift resshift && \
    git clone https://github.com/Pe4enIks/EMT && \
    mv EMT emt

Cloning into 'BasicSR'...
Cloning into 'Real-ESRGAN'...
Cloning into 'ResShift'...
Updating files: 100% (312/312), done.
Cloning into 'EMT'...


In [ ]:
# зависимости берем из requirements.txt, потому что через poetry нет возможности
%pip install -r requirements.txt

In [ ]:
# скрипт скачивания весов с yandex disk
import io
import os.path
import zipfile
from pathlib import Path
from urllib.parse import urlencode

import requests


def download_file(pk: str, local_name: str, src_type: str) -> None:
    """
    Скачивание единицы данных через API Yandex Disk.

    Parameters
    ----------
    pk : str
        Публичная ссылка на папку / файл на диске.
    local_name : str
        Путь для сохранения папки / путь к расположению файла (с именем файла).
    src_type : str
        Тип источника, folder / file.

    Returns
    -------
    None
    """
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"

    final_url = base_url + urlencode(dict(public_key=pk))
    response = requests.get(final_url)
    download_url = response.json()["href"]

    download_response = requests.get(download_url)
    if src_type == "file":
        with open(local_name, "wb") as f:
            f.write(download_response.content)
            print(f"File {local_name} downloaded")
    elif src_type == "folder":
        zip_file = zipfile.ZipFile(io.BytesIO(download_response.content))
        zip_file.extractall(local_name)
        print("Folder downloaded")


def download(download_source: str, save_folder: str) -> None:
    """
    Скачивание данных через API Yandex Disk.

    Parameters
    ----------
    download_source : str
        Url к скачиваемому источнику.
    save_folder : str
        Путь к папке для сохранения папки с Yandex Disk.

    Returns
    -------
    None
    """
    if save_folder:
        save_folder = Path(save_folder)
        os.makedirs(save_folder, exist_ok=True)
    else:
        save_folder = Path.cwd().parent

    files = {
        save_folder: (download_source, "folder"),
    }

    print(f"Will be downloaded {len(files)} sources")
    for name, (src_link, src_type) in files.items():
        download_file(src_link, str(name), src_type)


download("https://disk.yandex.ru/d/P1w6Tis0cjoclQ", "dvc_data")
download("https://disk.yandex.ru/d/csDaOUeCKxQk7g", "dvc_data")

In [61]:
# для возможности получить доступ к приватному репозиторию huggingface
import huggingface_hub as hf_hub

hf_hub.login()

In [ ]:
# скачаем необходимые данные через huggingface api
!mkdir -p "datasets/tmp"

hf_hub.snapshot_download(
    repo_id="epishchik/super-resolution-games",
    repo_type="dataset",
    local_dir="datasets/tmp",
    cache_dir="hf_cache",
    local_dir_use_symlinks=True,
    allow_patterns=[
        "data/GameEngineData/*270p.tar.gz",
        "data/GameEngineData/*1080p.tar.gz",
    ],
)

In [ ]:
# скрипт формирования датасета в нужном формате
!cd utils && bash create_local_dataset.sh \
    /home/jupyter/work/resources/SR-Gaming-Bench/datasets \
    GameEngineData \
    270p \
    1080p \
    false